# Assignment #1: Data Compression

In this assignment we're going to write a simple data compressor in Scala. You will have the chance to practice:
* the Scala fundamentals
* object-oriented principles in Scala
* immutable data structures
* string manipulation
* data compression!

## Intro

There are smart ways to encode strings of text for space efficiency - and by extension, any piece of data, as we can split the data in one byte or two byte chunks.

Look at this piece of text: **BACADAEAFABBAAAGH**. Considering each character as one byte, this uncompressed text takes 17 bytes = 136 bits. If we take a look at the fact that it only contains characters from A to H, we can use 3 bits per character instead of 8:

||||
|---|---|---|---|
|A=000   |B=001   |C=010   |D=011   |
|E=100   |F=101   |G=110   |H=111   |

which brings the total space down to 51 bits.

A better, smarter way of encoding the characters is by taking a look at _how often each character occurs_ in the original text. What would this look like if more frequent characters took fewer bits and less frequent characters took more bits to encode? Example:

||||
|---|---|---|---|
|A=0 |B=100|C=1010|D=1011|
|E=1100|F=1101|G=1110|H=1111|

With this encoding, we need only 41 bits to encode the text, compressing the original size by more than 3 times!

**This is our goal for this assignment - how to find such an encoding and how to compress and decompress text.** We will implement a classic compression technique known as [Huffman encoding](https://en.wikipedia.org/wiki/Huffman_coding). We will guide you on how to define and implement the relevnt data structures and how to progress with encoding your own pieces of text.

## How this works

We'll go through the technique here, step by step. We'll use "BACADAEAFABBAAAGH" as an example.

### Building the Huffman data  structures

The algorithm works as follows:

**Step 1**. The text to encode is converted into pairs of (character, number of occurrences of that character). The idea of Huffman encoding is that more frequent characters will have shorter codes.

    "BACADAEAFABBAAAGH" = [{A|8}, {B|3}, {H|1}, {G|1}, {F|1}, {E|1}, {D|1}, {C|1}]
    read {H|1} as "H appears 1 times in the text"

**Step 2**. These pairs are sorted in ascending order by the frequency of their characters and stored as Huffman "nodes" in a binary tree. The nodes are all leaves for now. We store all the nodes into this sorted list as a primitive priority queue.

    result = [{H|1}, {G|1}, {F|1}, {E|1}, {D|1}, {C|1}, {B|3}, {A|8}]

**Step 3**. The two pairs with the lowest frequency are extracted from the queue and combined into a higher level Huffman tree node as the pair (the combined set of the two nodes' characters, the sum of the two nodes' frequencies).

    {H|1} + {G|1} = {HG|2}

**Step 4**. Insert the new Huffman node into the queue and keep the sorted property of the queue. Remember, the sorting criterion is the ascending order of the Huffman nodes' frequencies.

    queue = [{F|1}, {E|1}, {D|1}, {C|1}, {HG|2}, {B|3}, {A|8}]
    
    {HG|2} has
        {H|1}
        {G|1}

    ^^^ this indented style is how we'll format the nodes in the hierarchy:
        the greated the indent, the lower the nodes will be in the tree.
        first value {H|1} is the left node, {G|1} is the right node

**Repeat steps 3 and 4 until there is only one node in the queue**. That will be the root of the Huffman encoding tree.

    iteration:
    
        before: [{F|1}, {E|1}, {D|1}, {C|1}, {HG|2}, {B|3}, {A|8}]
        {F|1} + {E|1} = {FE|2}
        after: [{D|1}, {C|1}, {FE|2}, {HG|2}, {B|3}, {A|8}]
        
        {HG|2} has
            {H|1}
            {G|1}
            
        {FE|2} has
            {F|1}
            {E|1}

see how the next two nodes are combined into one, much like the previous iteration.

    iteration:
    
        before: [{D|1}, {C|1}, {FE|2}, {HG|2}, {B|3}, {A|8}]
        {D|1} + {C|1} = {DC|2}
        after: [{DC|2}, {FE|2}, {HG|2}, {B|3}, {A|8}]
        
        {HG|2} has
            {H|1}
            {G|1}
            
        {FE|2} has
            {F|1}
            {E|1}
            
        {DC|2} has
            {D|1}
            {C|1}

Same thing.

    iteration:
    
        before: [{DC|2}, {FE|2}, {HG|2}, {B|3}, {A|8}]
        {DC|2} + {FE|2} = {DCFE|4}
        after: [{HG|2}, {B|3}, {DCFE|4}, {A|8}]
        
        {HG|2} has
            {H|1}
            {G|1}
        {DCFE|4} has
            {FE|2} which has
                {F|1}
                {E|1}
            {DC|2} which has
                {D|1}
                {C|1}

Now here, we have two higher nodes combined into an even higher node with 4 characters in it.

    iteration:
    
        before: [{HG|2}, {B|3}, {DCFE|4}, {A|8}]
        {HG|2} + {B|3} = {HGB|5}
        after: [{DCFE|4}, {HGB|5}, {A|8}]
        
        {HGB|5} has
            {B|3} and
            {HG|2} which has
                {H|1}
                {G|1}
        {DCFE|4} has
            {FE|2} which has
                {F|1}
                {E|1}
            {DC|2} which has
                {D|1}
                {C|1}

This iteration is with a higher node and a simple node. Look how the tree is starting to take shape.

    iteration:
    
        before: [{DCFE|4}, {HGB|5}, {A|8}]
        {DCFE|4} + {HGB|5} = {DCFEHGB|9}
        after: [{A|8}, {DCFEHGB|9}]
        
        {DCFEHGB|9} has 
            {DCFE|4} which has
                {FE|2} which has
                    {F|1}
                    {E|1}
                {DC|2} which has
                    {D|1}
                    {C|1}
            {HGB|5} which has
                {B|3} and
                {HG|2} which has
                    {H|1}
                    {G|1}

Almost there, just one more merge:

        before:[{A|8}, {DCFEHGB|9}]
        {DCFEHGB|9} + {A|8} = {DCFEHGBA|17}
        after: [{DCFEHGBA|17}]

        {DCFEHGBA|17} has
            {A|9} and
            {DCFEHGB|9} which has 
                {DCFE|4} which has
                    {FE|2} which has
                        {F|1}
                        {E|1}
                    {DC|2} which has
                        {D|1}
                        {C|1}
                {HGB|5} which has
                    {B|3} and
                    {HG|2} which has
                        {H|1}
                        {G|1}

And the Huffman tree is complete as the queue only has one node left. That will be the root of the tree.


### Encoding a character

Now that we have the Huffman tree set, encoding a character means traversing the Huffman tree. Let's encode "AB".

**Step 1.** Start from the root of the Huffman tree and with the empty string as the encoding.

        A = ""
        
        current node:
        {DCFEHGBA|17}
            {A|9}
            {DCFEHGB|9} 
                {DCFE|4} 
                    {FE|2}
                        {F|1}
                        {E|1}
                    {DC|2}
                        {D|1}
                        {C|1}
                {HGB|5}
                    {B|3}
                    {HG|2}
                        {H|1}
                        {G|1}

**Step 2.** Find which child (left or right) of this node contains the desired character. If it's the left child, append "0" to the encoding. If it's the right child, append "1". 

        A = "1"
        
**Step 3.** Move to the new node in the Huffman tree.

        current node: {A|9}
        
**Step 4.** Repeat steps 2 and 3 until you are at a leaf node. 

        (nothing to do)
        
**Step 5.** Write down the encoding, move to the root node again and go back to step 1 for the next character in your text.

        A = "1"
        
    iteration:
    
        B = "", current node:
        
        {DCFEHGBA|17}
            {A|9}
            {DCFEHGB|9} 
                {DCFE|4} 
                    {FE|2}
                        {F|1}
                        {E|1}
                    {DC|2}
                        {D|1}
                        {C|1}
                {HGB|5}
                    {B|3}
                    {HG|2}
                        {H|1}
                        {G|1}

    iteration:
    
        B = "1", current node:
        {DCFEHGB|9} 
            {DCFE|4} 
                {FE|2}
                    {F|1}
                    {E|1}
                {DC|2}
                    {D|1}
                    {C|1}
            {HGB|5}
                {B|3}
                {HG|2}
                    {H|1}
                    {G|1}
                        
    iteration:
    
        B = "11", current node:
        {HGB|5}
            {B|3}
            {HG|2}
                {H|1}
                {G|1}
                
    iteration:
    
        B = "110", current node: {B|3}, stop.


### Decoding text

The binary sequence will be essentially the way that you will need to traverse the Huffman tree to _retrieve_ the characters. 

**Step 1.** Start from the root node in the Huffman tree.

**Step 2.** Follow the binary sequence in your encoded text, bit by bit. If the first remaining bit in the encoded string is a 0, go left in the Huffman tree. If it's a 1, go right.

**Step 3.** Once you reach a leaf node, write down the letter contained within and go back to step 1 and carry on with the remaining binary string.

Example: say we want back the text "AB" from the binary encoding 0110.

        coding: 0110
        tree:
        {DCFEHGBA|17}
            {A|9}
            {DCFEHGB|9} 
                {DCFE|4} 
                    {FE|2}
                        {F|1}
                        {E|1}
                    {DC|2}
                        {D|1}
                        {C|1}
                {HGB|5}
                    {B|3}
                    {HG|2}
                        {H|1}
                        {G|1}
                        
       0 = left, currently at {A|9}
       got A
       1 = right, currently at {DCFEHGB|9}
       1 = right, currently at {HGB|5}
       0 = left, currently at {B|3}
       got B

       result: AB

## Part 1: the basic data structures

We'll define here a simple data structure that will allow us to find a good encoding for Huffman compression. We'll define a small "node" data structure (we'll call it `HuffmanNode`) which contains two fields: `chars`, which will be a `String`, and `occurrences`, which will be an `Int`. Define this below. We're going to expand this definition (and redefine `HuffmanNode`) as we go along.

In [34]:
/*
    YOUR CODE HERE
    
    Define a simple HuffmanNode class with two fields: chars = a String, and occurrences = an Int.
*/

In [35]:
// TEST
val node = new HuffmanNode("ABC", 3)
println(node.chars)
println(node.occurrences)

ABC
3


node: HuffmanNode = {ABC|3}

**Expected result:**<br/>

    ABC
    3

Now we need to make these nodes comparable by their occurrences values. Define a trait called `HuffmanComparable` with two methods:
* a method `absoluteValue` which returns and Int - leave that abstract
* a method `compareTo` which receives a `HuffmanComparable` - this will compare the absolute values

In [36]:
trait HuffmanComparable {
    // YOUR CODE HERE
    // a method absoluteValue
    // a method compareTo
}

defined trait HuffmanComparable

Now let's redefine `HuffmanNode` by taking into account all the properties described in the technique. Make `HuffmanNode` have two more fields (for the left and right child nodes) and make it extend the trait you just defined.

**Optional:** If you find it valuable, define a companion for `HuffmanNode` with one (or more) `apply` factory methods - they might come in handy later.

In [36]:
class HuffmanNode {

    // YOUR CODE HERE - add all the fields, make it extend the HuffmanComparable trait, and implement the methods

    // we added this for pretty printing - toString is a special method in the JVM
    override def toString: String = "{" + chars + "|" + occurrences + "}"
}

// YOUR CODE HERE - OPTIONAL - add a companion here


// TEST code
val nodeA = new HuffmanNode("A", 2)
val nodeB = new HuffmanNode("B", 3)
val comparison = nodeA compareTo nodeB
val parent = new HuffmanNode("AB", 5, nodeA, nodeB)

cmd36.sc:13: too many arguments for constructor HuffmanNode: ()Helper.this.HuffmanNode
val nodeA = new HuffmanNode("A", 2)
            ^cmd36.sc:14: too many arguments for constructor HuffmanNode: ()Helper.this.HuffmanNode
val nodeB = new HuffmanNode("B", 3)
            ^cmd36.sc:16: too many arguments for constructor HuffmanNode: ()Helper.this.HuffmanNode
val parent = new HuffmanNode("AB", 5, nodeA, nodeB)
             ^cmd36.sc:6: not found: value chars
    override def toString: String = "{" + chars + "|" + occurrences + "}"
                                          ^cmd36.sc:6: not found: value occurrences
    override def toString: String = "{" + chars + "|" + occurrences + "}"
                                                        ^

: 

## Part 2: The priority queue

Here is the following trait

In [38]:
trait HuffmanPQ {
    def add(node: HuffmanNode): HuffmanPQ
    def head: HuffmanNode
    def tail: HuffmanPQ
    def pop: HuffmanPQ
    def listElements: String // this will list all elements in a comma-separated string
    
    // we wrote this for pretty printing, no need to change
    override def toString: String = "[" + listElements + "]"
}

defined trait HuffmanPQ

Let's extend this trait by providing an empty queue implementation and a non-empty implementation.

In [38]:
object EmptyPQ extends HuffmanPQ {
    // YOUR CODE HERE implement the abstract methods 
}

class ConsPQ(/* YOUR CODE HERE members if you need them */) extends HuffmanPQ {
    // YOUR CODE HERE override the abstract methods
}

cmd38.sc:1: object creation impossible, since:
it has 5 unimplemented members.
/** As seen from object EmptyPQ, the missing signatures are as follows.
 *  For convenience, these are usable as stub implementations.
 */
  def add(node: cmd38Wrapper.this.cmd37.cmd15.wrapper.HuffmanNode): cmd38Wrapper.this.cmd37.wrapper.HuffmanPQ = ???
  def head: cmd38Wrapper.this.cmd37.cmd15.wrapper.HuffmanNode = ???
  def listElements: String = ???
  def pop: cmd38Wrapper.this.cmd37.wrapper.HuffmanPQ = ???
  def tail: cmd38Wrapper.this.cmd37.wrapper.HuffmanPQ = ???

object EmptyPQ extends HuffmanPQ {
       ^cmd38.sc:5: class ConsPQ needs to be abstract, since:
it has 5 unimplemented members.
/** As seen from class ConsPQ, the missing signatures are as follows.
 *  For convenience, these are usable as stub implementations.
 */
  def add(node: cmd38Wrapper.this.cmd37.cmd15.wrapper.HuffmanNode): cmd38Wrapper.this.cmd37.wrapper.HuffmanPQ = ???
  def head: cmd38Wrapper.this.cmd37.cmd15.wrapper.HuffmanNode =

: 

In [27]:
val empty = EmptyPQ
val oneElement = empty.add(HuffmanNode("A", 2))
val anotherElement = oneElement.add(HuffmanNode("B", 4))
val elementInBetween = anotherElement.add(HuffmanNode("C", 3))

empty: EmptyPQ.type = []
oneElement: HuffmanPQ = [{A|2}, ]
anotherElement: HuffmanPQ = [{A|2}, {B|4}, ]
elementInBetween: HuffmanPQ = [{A|2}, {C|3}, {B|4}, ]

**Expected result:**

    empty: EmptyPQ = []
    oneElement: HuffmanPQ = [{A|2}]
    anotherElement: HuffmanPQ = [{A|2}, {B|4}]
    elementInBetween: HuffmanPQ = [{A|2}, {C|3}, {B|4}]

## Part 3: utility functions

First step is counting the occurrences of characters in a text. For simplicity, we'll assume the text to be already sorted alphabetically for now. So the text "ABCABABC" will be sorted as "AAABBCC". The first function to implement is to check which index in the string has a **different** character than the one we are searching. In the "AAABBCC" example, if we're searching for A, the first index we're looking for is 3. 

In [38]:
import scala.annotation.tailrec

@tailrec
def firstIndexWithDifferentCharacter(text: String, char: Char, lastCheckedIndex: Int = 0): Int = {
    // YOUR IMPLEMENTATION HERE
}

cmd38.sc:4: type mismatch;
 found   : Unit
 required: Int
def firstIndexWithDifferentCharacter(text: String, char: Char, lastCheckedIndex: Int = 0): Int = {
                                                                                                 ^

: 

Next, implement a function which creates a `HuffmanPQ` from a text, which is still assumed to be sorted. A possible idea:
* start at the first character, count how many occurrences of that character you have (with the utility function above)
* create a `HuffmanNode` out of it and continue processing the rest of the text
* create a `HuffmanPQ` out of all the `HuffmanNode`s you have

In [38]:
def createHuffmanPQ(text: String): HuffmanPQ = {
    // YOUR CODE HERE
}

cmd38.sc:1: type mismatch;
 found   : Unit
 required: cmd38Wrapper.this.cmd37.wrapper.HuffmanPQ
def createHuffmanPQ(text: String): HuffmanPQ = {
                                               ^

: 

In [41]:
// TEST
val testText = "BACADAEAFABBAAAGH"
// notice that we're sorting the string for you
val testPQ = createHuffmanPQ(testText.sorted)

testText: String = "BACADAEAFABBAAAGH"
testPQ: wrapper.cmd29.cmd25.wrapper.HuffmanPQ = [{H|1}, {G|1}, {F|1}, {E|1}, {D|1}, {C|1}, {B|3}, {A|8}, ]

Expected result:
    
    testText: String = "BACADAEAFABBAAAGH"
    testPQ: HuffmanPQ = [{H|1}, {G|1}, {F|1}, {E|1}, {D|1}, {C|1}, {B|3}, {A|8}]

Now, implement a function to process the `HuffmanPQ` into a single `HuffmanNode` that contains the whole Huffman tree. Because all the `HuffmanNodes` are in ascending order of their frequency, all you have to do is to
* extract two nodes out of the queue
* combine them into one
* insert the resulted node back into the queue
* repeat the above until there's only one node left

In [39]:
def getFinalHuffmanNode(queue: HuffmanPQ): HuffmanNode = {
    // YOUR CODE HERE
}

cmd39.sc:1: type mismatch;
 found   : Unit
 required: cmd39Wrapper.this.cmd15.wrapper.HuffmanNode
def getFinalHuffmanNode(queue: HuffmanPQ): HuffmanNode = {
                                                         ^

: 

In [40]:
// TEST
val huffmanTree = getFinalHuffmanNode(testPQ)

huffmanTree: HuffmanNode = {AHGFEDCB|17}

**Expected result** (characters may appear in a slightly different order):

    huffmanTree: HuffmanNode = {AHGFEDCB|17}


## Part 4: Compressing text

Good! You have a Huffman tree - now onto compressing text! As the algorithm states, compressing means traversing the tree until you've reached a leaf node, keeping track of the directions you chose in the tree (0 = left, 1 = right).

In [34]:
def compress(text: String, huffmanTree: HuffmanNode): String = {
    
    /*
        YOUR CODE HERE.
        
        Hint: you might want to define an auxiliary function to encode a single character first.
    */
}

val compressedText = compress(testText, huffmanTree)
val newLength = compressedText.length

defined function compress
compressedText: String = "11101101011000101101010011111100010011000"
newLength: Int = 41

**Expected result:**

    compressedText: String = "11101101011000101101010011111100010011000"
    newLength: Int = 41

## Part 5: Decompressing text

Decompressing should be even easier! At every step, look at the first bit in the remaining encoded string - that will tell you which next node in the Huffman tree to choose.

In [39]:
/* 
    YOUR CODE HERE - define a function to decompress.
    Advice: use auxiliary parameters as accumulators for any "state" you might want to keep.
    Examples: the current character in the encoded string, the current node in your traversal etc
*/

val decompressedText = // TODO call your decompress function here
val isTheSame = decompressedText.equals(testText)

: 

**Expected result:**

    decompressedText: String = "BACADAEAFABBAAAGH"
    isTheSame: Boolean = true